<h1 style=\"text-align: center; font-size: 50px;\">😷 A tale of two cities analyzing trends </h1>
This notebook shows an visual data analysis of the effects of COVID-19 in two different cities: New York and London

## Notebook Overview
- Imports
- Configurations
- Preparing the Data
- Univariate Analysis
- Bivariate Analysis
- Correlation of features
- Time-series Decomposition
- Exponential Smoothing Forecasting Methods
- Vector Autoregression (VAR) 
- Cointegration Test
- Stationarity of a Time-Series
- Training the VAR model
- Autocorrelation of Residuals 
- Forecasting
- Model Evaluation
- Logging Model to MLflow
- Fetching the Latest Model Version from MLflow
- Loading the Model and Running Inference

## Imports

In [ ]:
# ------------------------ Data Manipulation ------------------------
import pandas as pd
import numpy as np

# ------------------------ Visualization Libraries ------------------------ 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# ------------------------ System Utilities ------------------------
import warnings
import logging
from pathlib import Path
import os

# ------------------------ Statistical and Time Series Analysis ------------------------
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.stats.stattools import durbin_watson
from statsmodels.tsa.stattools import acf
from statsmodels.tools.sm_exceptions import ConvergenceWarning, ValueWarning

# ------------------------ MLflow for Experiment Tracking and Model Management ------------------------
import mlflow
from mlflow import MlflowClient
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec, TensorSpec, ParamSchema, ParamSpec


## Configurations

In [ ]:
# Plot configurations
plt.style.use('fivethirtyeight')
params = {'legend.fontsize': 'x-large',
         'figure.figsize': (20,10),
         'axes.labelsize': 25,
         'xtick.labelsize':25,
         'ytick.labelsize':25,
        'font.family':'serif'}
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

In [ ]:
# Suppress Python warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.simplefilter('ignore', ValueWarning)

In [ ]:
# Create logger
logger = logging.getLogger("cities_analysis_logger")
logger.setLevel(logging.INFO)

formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s", 
                              datefmt="%Y-%m-%d %H:%M:%S")  

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.propagate = False

In [ ]:
# ------------------------- Paths -------------------------
DATA_PATH = "/home/jovyan/datafabric/tutorial/"

# ------------------------ MLflow Integration ------------------------
EXPERIMENT_NAME = "Two_Cities_Experiment"
RUN_NAME = "Two_Cities_Run"
MODEL_NAME = "Two_Cities_Model"

In [ ]:
logger.info('Notebook execution started.')

## Verify Assets


In [ ]:
# Check whether the Dataset file exists
is_dataset_available = Path(DATA_PATH).exists()

# Log the configuration status of the dataset
if is_dataset_available:
    logger.info("The Dataset is properly configured.")
else:
    logger.info(
        "The Dataset is not properly configured. Please create and download the required assets "
        "in your project on AI Studio."
    )


## Preparing the Data

### Acknowledgments:
I'd like to thank the original authors of these data sources!

| Data | Original Source |
| --- | --- |
| Mobility Data | [COVID-19 Community Mobility Reports](https://www.google.com/covid19/mobility/) |
| NYC Cases | [NYC Department of Health and Mental Hygiene](https://www1.nyc.gov/site/doh/index.page) |
| London Cases | [GOV.UK Coronavirus (COVID-19) in the UK](https://coronavirus.data.gov.uk/) |

In [ ]:
source_folder = DATA_PATH
ny_mobility = pd.read_csv(f"{source_folder}/NewYork_mobility.csv")
ldn_mobility = pd.read_csv(f"{source_folder}London_mobility.csv")
ny_cases = pd.read_csv(f"{source_folder}daily_data_NewYork.csv")
ldn_cases = pd.read_csv(f"{source_folder}daily_data_London.csv")

In [ ]:
def rename_mobility_cols(df):
    """
    Renames the and cleans the dataset columns.

    Parameters:
        df(pd.DataFrame): A Dataframe containing the data.

    Returns:
        pd.DataFrame: The cleaned dataframe with the renamed columns.
    """
    # Rename columns
    df = df.rename(columns={'country_region':'country'})
    df = df.rename(columns={'retail_and_recreation_percent_change_from_baseline':'retail'})
    df = df.rename(columns={'grocery_and_pharmacy_percent_change_from_baseline':'pharmacy'})
    df = df.rename(columns={'parks_percent_change_from_baseline':'parks'})
    df = df.rename(columns={'transit_stations_percent_change_from_baseline':'transit_station'})
    df = df.rename(columns={'workplaces_percent_change_from_baseline':'workplaces'})
    df = df.rename(columns={'residential_percent_change_from_baseline':'residential'})
    df.drop(['country_region_code','sub_region_1', 'sub_region_2', 'residential'], axis=1, inplace = True)
    return df


ny_mobility = ny_mobility.loc[ny_mobility['sub_region_2'] == "New York County"].reset_index(drop=True)
ldn_mobility = ldn_mobility.loc[ldn_mobility['sub_region_2'] == "City of London"].reset_index(drop=True)

ny_mobility = rename_mobility_cols(ny_mobility)
ldn_mobility = rename_mobility_cols(ldn_mobility)

mobility_features = ny_mobility.columns[6:]

| Mobility Features     | Description                                                                                                                           |
|-----------------|---------------------------------------------------------------------------------------------------------------------------------------|
| country          | Country Name                                                                         |
| metro_area       | Metropolitan area                                                                    |
| iso_3166_2_code  | Codes for the names of the principal subdivisions (e.g. provinces or states)         |
| census_fips_code | Census fips code                                                                     |
| place_id         | Place IDs uniquely identify a place in the Google Places database and on Google Maps |
| date             | Date                                                                                 |
| retail          | Mobility trends for places like restaurants, cafes, shopping centers, theme parks, museums, libraries, and movie theaters.            |
| pharmacy        | Mobility trends for places like grocery markets, food warehouses, farmers markets, specialty food shops, drug stores, and pharmacies. |
| parks           | Mobility trends for places like local parks, national parks, public beaches, marinas, dog parks, plazas, and public gardens.          |
| transit_station | Mobility trends for places like public transport hubs such as subway, bus, and train stations.                                        |
| workplaces      | Mobility trends for places of work.                                                                                                   |

In [ ]:
ldn_mobility.head()

Try it out yourself 🚀 : [Get the Address for a Place ID 🌎](https://developers.google.com/maps/documentation/javascript/examples/geocoding-place-id)

![](https://i.imgur.com/B69162k.png)

In [ ]:
def rename_dailyData_cols(df):
    """
    Renames the daily data DataFrame columns to standardized names.

    Parameters:
        df (pd.DataFrame): A Dataframe containing daily COVID-19 statistics.

    Returns:
        pd.DataFrame: The Dataframe with the renamed columns.
    """
    mapping = {df.columns[0]:'date', df.columns[1]: 'case_count', df.columns[2]:'hospitalized_count', df.columns[3]: 'death_count'} 
    df = df.rename(columns = mapping)
    return df

ny_cases = ny_cases[['date_of_interest','CASE_COUNT','HOSPITALIZED_COUNT','DEATH_COUNT']]
ldn_cases = ldn_cases[['date','newCasesBySpecimenDate', 'newAdmissions', 'newDeaths28DaysByDeathDate']]

ny_cases = rename_dailyData_cols(ny_cases)
ldn_cases = rename_dailyData_cols(ldn_cases)

cases_features = ny_cases.columns[1:]

| Cases Features     | Description                    |
|--------------------|--------------------------------|
| date               | Date                           |
| case_count         | Number of daily cases recorded |
| hospitalized_count | Number of people hospitalized  |
| death_count        | Number of deaths recorded      |

In [ ]:
ny_cases.head()

In [ ]:
for df in ny_mobility, ldn_mobility, ny_cases, ldn_cases:
    df['date'] = df['date'].astype('datetime64[ns]')

In [ ]:
def merge_data(mobility, cases):
    """
    Merges mobility and case data on the 'date' column using an inner join.

    Parameters:
        mobility (pd.DataFrame): DataFrame containing mobility indicators.
        cases (pd.DataFrame): DataFrame containing COVID-19 case statistics.

    Returns:
        pd.DataFrame: Merged DataFrame containing both mobility and case data, aligned by date.
    """
    merged_df = pd.merge(mobility, cases, how='inner', on = 'date')
    return merged_df

# setting date as the index column
ny_df = merge_data(ny_mobility, ny_cases).set_index('date')
ldn_df = merge_data(ldn_mobility, ldn_cases).set_index('date')
ldn_df = ldn_df.iloc[:-2,:]

features = ny_df.columns[5:]

In [ ]:
ny_df.head()

In [ ]:
ldn_df.head()

In [ ]:
# during and after the pandemic dates 
start_date = min(ny_df.index.min(), ldn_df.index.min()).strftime('%Y-%m-%d')
split_date = '2022-02-01'
end_date = max(ny_df.index.max(), ldn_df.index.max()).strftime('%Y-%m-%d')

## Univariate Analysis

In [ ]:
def univariate_trends(idx): 
    """
    Plots the time series trend of a single mobility or case feature for both New York and London.

    Parameters:
        idx (int): Index of the feature to plot, based on the 'features' list.
    """
    plt.rcParams.update(params)          
    ax = sns.lineplot(x=ny_df.index, y=features[idx], data=ny_df)
    ax = sns.lineplot(x=ldn_df.index, y=features[idx], data=ldn_df)
    
    text_pos_x1 = ax.get_xlim()[0] + 300
    text_pos_x2 = ax.get_xlim()[1] - 175
    text_pos_y = ax.get_ylim()[1]
    plt.text(x=text_pos_x1, y=text_pos_y, s='COVID era', alpha=0.7, color='#334f8d', size = 25, weight='bold')
    plt.text(x=text_pos_x2, y=text_pos_y, s='Post COVID era', alpha=0.7, color='#334f8d', size = 25, weight='bold')
    
    plt.axvspan(start_date, split_date, color='y', alpha=0.1, lw=0)
    plt.axvspan(split_date, end_date, color='g', alpha=0.1, lw=0)
    
    plt.suptitle("Change in " + features[idx] + " over time", size=30, color='#334f8d', weight="bold")
    
    plt.legend(labels=['New York', 'London'], bbox_to_anchor=(1, 1), loc='upper left', borderaxespad=0)
    plt.show()

In [ ]:
#trends over time
for idx in range(len(features)):
    univariate_trends(idx)

## Bivariate Analysis

In [ ]:
def bivariate_trends(df, idx1, idx2, place):
    """
    Plots a dual-axis time series chart showing the relationship between two features over time for a given location.

    Parameters:
        df (pd.DataFrame): The DataFrame for the selected city.
        idx1 (int): Index of the first feature to plot.
        idx2 (int): Index of the second feature to plot.
        place (str): Name of the city/region.
    """
    colors = ["#ff9e00", "#a6808c","#70a38d", "#6665dd", "#b57ba6"]
    color1 = colors[idx1]
    if place=='New York': 
        color2 = "#30a2da" 
    else: 
        color2 = "#fc4f30"
         
    plt.rcParams.update(params)
    fig, ax = plt.subplots()
    
    sns.lineplot(x=df.index, y=features[idx1], data=df, color=color1, ax=ax)
    ax.tick_params(axis='y', labelcolor=color1)

    ax2 = ax.twinx()
    sns.lineplot(x=df.index, y=features[idx2], data=df, color=color2, ax=ax2)
    ax2.tick_params(axis='y', labelcolor=color2)
    
    text_pos_x1 = ax.get_xlim()[0] + 300
    text_pos_x2 = ax.get_xlim()[1] - 175
    text_pos_y = ax2.get_ylim()[1]
    plt.text(x=text_pos_x1, y=text_pos_y, s='COVID era', alpha=0.7, color='#334f8d', size = 25, weight='bold')
    plt.text(x=text_pos_x2, y=text_pos_y, s='Post COVID era', alpha=0.7, color='#334f8d', size = 25, weight='bold')
    
    plt.axvspan(start_date, split_date, color='y', alpha=0.1, lw=0)
    plt.axvspan(split_date, end_date, color='g', alpha=0.1, lw=0)
    
    plt.legend(labels = [place,features[idx1]], bbox_to_anchor=(1, 1.2), loc='upper left', borderaxespad=0, facecolor="white")
    leg = ax2.get_legend()
    leg.legendHandles[1].set_color(color1)
    leg.legendHandles[1].set_alpha(1)
    
    plt.suptitle(place + " : Change in " + features[idx1] + " and "  + features[idx2] + " over time", size=30, color='#334f8d', weight="bold")
    plt.show()

In [ ]:
for idx in range(len(mobility_features)):
    bivariate_trends(ny_df,idx, 5, "New York")
    bivariate_trends(ldn_df,idx, 5, "London")

## Correlation of features

In [ ]:
def plot_correlation(df, place):
    """
    Plots side-by-side heatmaps of the correlation matrices of selected features for two
    different time periods: during COVID and post-COVID.

    Parameters:
        df (pd.DataFrame): DataFrame with time-indexed features for a specific region.
        place (str): Name of the city/region to be used in the plot title.
    """
    plt.rcParams.update(params)
    fig, ax = plt.subplots(1,2)
    df_during = df.loc[df.index < split_date]
    df_post = df.loc[df.index >= split_date]
    
    corr1=df_during[features].corr()
    corr2=df_post[features].corr()
    
    mask1 = np.triu(np.ones_like(corr1, dtype=bool))
    mask2 = np.triu(np.ones_like(corr2, dtype=bool))
    
    fig.suptitle("Correlation of features in " + place, fontsize = 30)
    sns.set_style("darkgrid")
    sns.heatmap(corr1, mask=mask1, cmap='BuPu', 
                square=True, linewidths=.5,annot=True, fmt='.2f', cbar=False, ax=ax[0])
    sns.heatmap(corr2, mask=mask2, cmap='BuPu', 
                square=True, linewidths=.5,annot=True, fmt='.2f', cbar=False, ax=ax[1])
    
    ax[0].title.set_text('COVID era')
    ax[1].title.set_text('Post COVID era')
    
    plt.tight_layout()
    plt.show()

In [ ]:
#correlation of features
plot_correlation(ny_df, "New York")
plot_correlation(ldn_df, "London")

📌 **Mobility Trend Attributes**
- **COVID era**: Strong pair correlations indicating that the global lockdown restrictions resulted in a drop in visits in all areas.
- **Post COVID era**: Fall in pair correlations indicating that the trends are changing.

📌 **Cases Attributes**
- **COVID era**: Strong pair correlations between death count and hospitalized count.
- **Post COVID era**: Fall in pair correlation of death count and hospitalized count indicating that the trends are changing.

## Time-series Decomposition

Time series Decomposition helps us to analyze data as a combination of level, trend, seasonality and noise components.

How does this help us? 🤨
- Allows us to focus on predicting the general trend of the data 📈
- Reveals note-worthy behavior in the seasonal component ☀️🍂❄️🌱

In [ ]:
def time_series_plot(df, col, period):
    """
    Performs and visualizes seasonal decomposition of time series into trend,
    seasonal and residual components.

    Parameters:
        df (pd.DataFrame): Time-indexed DataFrame containing the time siries.
        col (str): Name of the column to decompose.
        period (int): Number of the observations per cycle.
    """
    decomposition = seasonal_decompose(df[col], period = period)
    
    plt.rcParams.update(params)
    figure = decomposition.plot()
    plt.show()

In [ ]:
print("\n\n"+color.BOLD + color.PURPLE + 'New York:' + color.END)
time_series_plot(ny_df, cases_features[0],12)
time_series_plot(ny_df, cases_features[0],52)

In [ ]:
class Tweet(object):
    def __init__(self, embed_str=None):
        self.embed_str = embed_str

    def _repr_html_(self):
        return self.embed_str

s = ("""
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">New York state reports highest number of daily Covid cases of entire pandemic at more than 21,000 <a href="https://t.co/0VIqv8lu5G">https://t.co/0VIqv8lu5G</a></p>&mdash; CNBC (@CNBC) <a href="https://twitter.com/CNBC/status/1471945248419074055?ref_src=twsrc%5Etfw">December 17, 2021</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
""")

Tweet(s)

In [ ]:
print("\n\n"+color.BOLD + color.PURPLE + 'London:' + color.END)
time_series_plot(ldn_df, cases_features[0],12)
time_series_plot(ldn_df, cases_features[0],52)

## Exponential Smoothing Forecasting Methods

> A family of forecasting models focused on utilizing **weighted averages of past observations to forecast new values**. 
>
> These methods are also called ETS models, referring to the explicit modeling of **Error, Trend and Seasonality**. 

In [ ]:
def simple_exp_smoothing(df, col, triple=False):
    """
    Applies Simple Exponential Smoothing (SES) to a time series and plots the original data,
    the fitted values, and a 52-period forecast.

    Parameters:
        df (pd.DataFrame): Time-indexed DataFrame with the target series.
        col (str): Name of the column.
        triple (bool): If 'True', aplies SES with three smoothing levels (alpha = 0.2, 0.5, 0.9) for comparison.
                       If 'False', uses only alpha = 0.5.
    """
    df = df[[col]]
    plt.rcParams.update(params)
    if triple == True:
        for alpha_sm, color in zip([0.2, 0.5, 0.9], ['red', 'orange', 'brown']):

            df.plot.line()

            fit1 = SimpleExpSmoothing(df).fit(smoothing_level = alpha_sm  ,optimized=False)
            fcast1 = fit1.forecast(52).rename('alpha = ' + str(alpha_sm))
            fcast1.plot(marker='o', color=color, legend=True)
            fit1.fittedvalues.plot(color=color)

            plt.show()
    else:
            df.plot.line()
            color = 'orange'
            fit1 = SimpleExpSmoothing(df).fit(smoothing_level = 0.5  ,optimized=False)
            fcast1 = fit1.forecast(52).rename('alpha = 0.5')
            fcast1.plot(marker='o', color=color, legend=True)
            fit1.fittedvalues.plot(color=color)

            plt.show()
        
def double_exp_smoothing(df, col):
    """
    Applies Double Exponential Smoothing to capture level and trend in the time series, and plots the original data,
    the fitted values, and a 52-period forecast.

    Parameters:
        df (pd.DataFrame): Time-indexed DataFrame with the target series.
        col (str): Name of the column.
    """
    df = df[[col]]
    plt.rcParams.update(params)
    
    df.plot.line()
    fit1 = Holt(df).fit(smoothing_level=0.5, smoothing_slope=0.5, optimized=False)
    fcast1 = fit1.forecast(52).rename("Double Exponential Smoothing")
    fit1.fittedvalues.plot(color='orange')
    fcast1.plot(color='orange', legend=True)

    plt.show()
    
def triple_exp_smoothing(df, col):
    """
    Applies Triple Exponential Smoothing to model trend and seasonality in time series, and plots the original data,
    the fitted values, and a 52-period forecast.

    Parameters:
        df (pd.DataFrame): Time-indexed DataFrame with the target series.
        col (str): Name of the column.
    """
    df = df[[col]]
    plt.rcParams.update(params)
    
    df.plot.line()
    fit1 = ExponentialSmoothing(df, seasonal_periods=12, trend='add', seasonal='add')
    fit1 = fit1.fit(smoothing_level=0.5)
    fit1.fittedvalues.plot(color='orange')
    fit1.forecast(52).rename("Triple Exponential Smoothing").plot(color='red', legend=True)

    plt.show()

## 1. Simple Exponential Smoothing
\begin{equation}
\hat  Y_{t+1} = \hat Y_t + \alpha  (Y_t - \hat Y_t)
\end{equation}

- Predicted value for time period t+1 = Predicted value for previous time period + Adjustment for the error made in predicting the previous period's value

- α can assume any value between 0 and 1.

- For small values of α, estimates change very slowly.

- For large values of α, estimates change rapidly, and our prediction is basically the most recent observation.

- As we move towards forecasting the next observed value and we run out of data points, we notice that the equation takes the form of:
\begin{equation}
\hat  Y_{t+1} = \hat Y_t 
\end{equation}


In [ ]:
simple_exp_smoothing(ny_df, "case_count", triple=True)


## 2. Double Exponential Smoothing (Holt's method)

Forecasting function:
\begin{equation}
\hat  Y_{t+n} = E_t + nT_t
\end{equation}

Computing the base level for time period t
\begin{equation}
E_t = \alpha Y_t + (1-\alpha)(E_{t-1} + T_{t-1})
\end{equation}

Computing the expected trend value for time period t
\begin{equation}
T_t = \beta (E_t - E_{t-1}) + (1-\beta)T_{t-1}
\end{equation}

- 0 <= α <= 1 and 0 <= β <= 1

Ideal for time series data with a linear trend.

In [ ]:
double_exp_smoothing(ny_df, "case_count")

## 3. Triple Exponential Smoothing (Holt-Winter's method)


Forecasting function:
\begin{equation}
\hat  Y_{t+n} = E_t + nT_t + S_{t+n-p}
\end{equation}

Computing the base level for time period t
\begin{equation}
E_t = \alpha (Y_t-S_{t-p}) + (1-\alpha)(E_{t-1} + T_{t-1})
\end{equation}

Computing the expected trend value for time period t
\begin{equation}
T_t = \beta (E_t - E_{t-1}) + (1-\beta)T_{t-1}
\end{equation}

Computing the expected seasonal factor for time period t
\begin{equation}
S_t = \gamma (Y_t - E_t) + (1-\gamma)S_{t-p}
\end{equation}

- 0 <= α <= 1 , 0 <= β <= 1 and 0 <= γ <= 1
- p represents the number of seasons in the time series

Applicable for time series data exhibiting a trend and seasonality component.

In [ ]:
print("\n\n"+color.BOLD + color.PURPLE + 'New York:' + color.END)
triple_exp_smoothing(ny_df, "case_count")

In [ ]:
print("\n\n"+color.BOLD + color.PURPLE + 'London:' + color.END)
triple_exp_smoothing(ldn_df, "case_count")

## Vector Autoregression (VAR) 

Multivariate forecasting algorithm that can be used when **two or more time series influence each other** ➡️⬅️ 

**Advantages**
- Ability to capture sophisticated real-world behaviour and interconnected dynamics of time series data ✅ 
- Improved forecasting efficiency ✅  

**Procedure**

Each variable is modeled as a **linear combination** of: 
- its own past values
- the past values of other variables in the model

## Cointegration Test

**Note**

>  ⚠️ Cointegration is not the same as correlation ⚠️
> 
> - Correlation measures whether two or more time-series variables move together in the long-run.
> - Cointegration measures whether the difference between their means remains constant or not.

In [ ]:
ny_data = ny_df.iloc[:,5:]
ldn_data = ldn_df.iloc[:,5:].dropna()

**Johansen's Test**

*To check if three or more time series are cointegrated*
- Null hypothesis: the time series are not cointegrated
- If the trace statistic is greater than the critical value, we reject the null hypothesis and accept the alternate hypothesis, suggesting that the series are cointegrated.

In [ ]:
def johansens_test(df, alpha=0.05): 
    """
    Performs Johansen's cointegration test on a set of time series and print the results
    including trace statistics, critical values, and whether the null hypothesis is rejected.

    Parameters:
        df (pd.DataFrame): DataFrame containing multiple time series.
        alpha (float): Significance level for the test. Defaults to 0.05.
    """
    result = coint_johansen(df,-1,1)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = result.lr1
    cvts = result.cvt[:, d[str(1-alpha)]]
    
    print("{: >20} {: >20} {: >20} {: >10}".format(*['Column name', 'Trace Statistic' , 'Critical Value(95%)', 'Null Hypothesis Rejected' ]))
    for col, trace, cvt in zip(df.columns, traces, cvts):
        ans = "Yes" if trace > cvt else "No"
        print("{: >20} {: >20} {: >20} {: >10}".format(*[col, round(trace,2) ,cvt, ans]))

def cointegration_test(data):
    """
    Splits a time series DataFrame into two periods and 
    performs Johansen's cointegration test on each subset, printing the results

    Parameters:
        data (pd.DataFrame): DataFrame with a time-index and multiple time series columns.
    """
    data_during = data.loc[data.index < split_date]
    data_post = data.loc[data.index >= split_date]

    print(color.BOLD + color.DARKCYAN + 'During the pandemic:' + color.END)
    johansens_test(data_during)
    print("\n\n"+color.BOLD + color.DARKCYAN + 'Post pandemic:' + color.END)
    johansens_test(data_post)

print("\n\n"+color.BOLD + color.PURPLE + 'New York:' + color.END)
cointegration_test(ny_data)
print("\n\n"+color.BOLD + color.PURPLE + 'London:' + color.END)
cointegration_test(ldn_data)

## Stationarity of a Time-Series

Before we apply the VAR model we need to ensure that all the time series variables in the data are stationary.

**Augmented Dickey-Fuller Test**

*To check if all the time series variables in the data are stationary*
- Null hypothesis: the time series is considered non-stationary
- If the p-value of the ADF test is less than the significance level then we reject the null hypothesis and accept the alternate hypothesis, considering that the time series is stationary.

In [ ]:
def adfuller_test(series, series_name):
    """
    Performs the Augmented Dickey-Fuller (ADF) Test to check for stationarity in a time series.
    Prints the p-value, significance level, and wether the null hypothesis (non-stationarity) is rejected.

    Parameters:
        series (pd.Series): The time series data to be tested.
        series_name (str): Name of the series, used for display purposes in the output
    """
    r = adfuller(series, autolag='AIC')
    result = {'ADF_test_statistic':round(r[0], 4), 'p-value':round(r[1], 4), 'num_lags':round(r[2], 4), 'num_observations':r[3]}
    
    significance_lvl=0.05
    p_value = result['p-value'] 
    
    ans = "Yes" if p_value <= significance_lvl else "No" 
    
    print("{:>18} {: >10} {: >15} {: >25} {: >10}".format(*[series_name, p_value, significance_lvl,  ans, ans ]))

In [ ]:
num_obs = 7
ny_train, ny_test = ny_data[0:-num_obs], ny_data[-num_obs:]
ldn_train, ldn_test = ldn_data[0:-num_obs], ldn_data[-num_obs:]

In [ ]:
def display_ADF(df):
    """
    Applies the Augmented Dickey-Fuller (ADF) Test to each time series in the DataFrame
    and prints a summary table showing the p-value, significance level, and wether the series is stationary.

    Parameters:
        df (pd.DataFrame): DataFrame where each column is a univariate time series to be tested.
    """
    print("{:>18} {: >10} {: >15} {: >25} {: >10}".format(*['Series', 'P-Value' ,'Significance Level', 'Null Hypothesis Rejected', 'Stationary' ]))
    for col_name, col_data in df.items():
        adfuller_test(col_data, col_name)

print("\n\n"+color.BOLD + color.PURPLE + 'New York:' + color.END)
display_ADF(ny_train)
print("\n\n"+color.BOLD + color.PURPLE + 'London:' + color.END)
display_ADF(ldn_train)

- Some series aren't stationary so we will be taking the first-order difference of the entire data.

- After this we will re-run the ADF test on each differenced series.

In [ ]:
ny_differenced = ny_train.diff().dropna()
ldn_differenced = ldn_train.diff().dropna()

print("\n\n"+color.BOLD + color.PURPLE + 'New York:' + color.END)
display_ADF(ny_differenced)
print("\n\n"+color.BOLD + color.PURPLE + 'London:' + color.END)
display_ADF(ldn_differenced)

All the series are now stationary!

## Training the VAR model

We select the order that gives a model with least BIC.

> BIC or Bayesian Information Criterion is a method for scoring and selecting a model. 

In [ ]:
ny_model = VAR(ny_differenced)
res = ny_model.select_order(maxlags=15)
res.summary()

In [ ]:
ldn_model = VAR(ldn_differenced)
res = ldn_model.select_order(maxlags=15)
res.summary()

Training the model 

In [ ]:
ny_model = ny_model.fit(7)
ldn_model = ldn_model.fit(4)

## Autocorrelation of Residuals 

Durbin Watson’s Statistic

*To check for autocorrelation in a regression model's output*
- Values range from 0 to 4, with a value of 2 indicating **zero autocorrelation**.
- Values below 2 indicate **positive autocorrelation**. 
- Values above 2 indicate **negative autocorrelation**.

In [ ]:
def DW(model, data):
    """
    Computes the Durbin Watson’s Statistic for residuals of a fitted time series model,
    assessing the presence of autocorrelation for each series in the dataset.

    Parameters:
        model: Fitted VAR or similar multivariate time series model with a '.resid' attribute.
        data (pd.DataFrame): Original time series data used in the model, used for column labels.
    """
    result = durbin_watson(model.resid)

    print("{:>18} {: >15}".format(*['Series', 'DW Statistic']))
    for col, val in zip(data.columns, result):
        print("{:>18} {: >15}".format(*[col, round(val, 2)]))
        
print("\n\n"+color.BOLD + color.PURPLE + 'New York:' + color.END)
DW(ny_model, ny_data)
print("\n\n"+color.BOLD + color.PURPLE + 'London:' + color.END)
DW(ldn_model, ldn_data)

- In the event that auto correlation exists, it undervalues the standard error and may cause us to believe that predictors are significant when in reality they are not.

- Since the values are closer to 2 in our case, we can safely proceed with the VAR model.

## Forecasting

In [ ]:
lag_order = ny_model.k_ar
ny_forecast_input = ny_differenced.values[-lag_order:]
ny_differenced[-lag_order:]

In [ ]:
lag_order = ldn_model.k_ar
ldn_forecast_input = ldn_differenced.values[-lag_order:]
ldn_differenced[-lag_order:]

In [ ]:
fc = ny_model.forecast(y=ny_forecast_input, steps=num_obs)
ny_forecast_output = pd.DataFrame(fc, index=ny_data.index[-num_obs:], columns=ny_data.columns + '_forecast')
ny_forecast_output

In [ ]:
fc = ldn_model.forecast(y=ldn_forecast_input, steps=num_obs)
ldn_forecast_output = pd.DataFrame(fc, index=ldn_data.index[-num_obs:], columns=ldn_data.columns + '_forecast')

Rolling back the 1st difference transformation

In [ ]:
def rolling_back_transformation(df_train, forecast_output):
    """
    Reconstructs the original forecated values by reversing first-order differencing.
    This is done by cumulatively summing the differenced forecasts and adding the last observed value
    from the training data for each series.

    Parameters:
        df_train (pd.DataFrame): Original training dataset, used to retrieve last known values.
        forecast_output (pd.DataFrame): Forecasted differenced values, with column names ending in '_forecast'.

    Returns:
        pd.DataFrame: Forecast with restored original scale, using cumulative sums.
    """
    forecast_final = forecast_output.copy()

    for col in df_train.columns:        
        forecast_final[str(col)+'_forecast'] = df_train[col].iloc[-1] + forecast_output[str(col)+'_forecast'].cumsum()
    
    return forecast_final

ny_forecast_final = rolling_back_transformation(ny_train, ny_forecast_output)
ldn_forecast_final =rolling_back_transformation(ldn_train, ldn_forecast_output)

ny_forecast_final

In [ ]:
def plot_forecast(place, col, df_test, forecast_final): 
    """
    Plots actual vs forecasted values for a specific time series feature in a given location.

    Parameters:
        place (str): Name of the location used in the plots title.
        col (str): Name of the column/feature to plot.
        df_test (pd.DataFrame): DataFrame containing actual observed values.
        forecast_final (pd.DataFrame): DataFrame containing forecasted values.
    """
    plt.rcParams.update(params)
    sns.lineplot(x=df_test.index, y=col, data=df_test, label='Actual')

    forecast_col = col+'_forecast'
    sns.lineplot(x=forecast_final.index, y=forecast_col, data=forecast_final, label='Forecast')
    
    plt.title(col + " in " + place)
    plt.tight_layout();
    plt.show()
    
for feature in features:
    plot_forecast("New York", feature, ny_test, ny_forecast_final)
    plot_forecast("London", feature, ldn_test, ldn_forecast_final)

## Model Evaluation

In [ ]:
def forecast_accuracy(df_test, forecast_final, col):
    """
    Calculates and prints forecast accuracy metrics for a given time series column.
    Metrics computed:
        - Mean Error (ME)
        - Mean Absolute Error (MAE)
        - Root Mean Squared Error (RMSE)

    Parameters:
        df_test (pd.DataFrame): DataFrame with actual observed values.
        forecast_final (pd.DataFrame): DataFrame containing forecasted values.
        col (str): Name of the column/feature to evaluate.
    """
    col_forecast = col +'_forecast'
    forecast = forecast_final[col_forecast].values
    actual = df_test[col]
    me = np.mean(forecast - actual)             # Mean Error
    mae = np.mean(np.abs(forecast - actual))    # Mean Absolute Error
    rmse = np.mean((forecast - actual)**2)**.5  # Root Mean Squared Error
    
    print("{:>18} {:>10} {: >20} {: >20}".format(*[col, round(me,3), round(mae,3), round(rmse,3)]))

def display_accuracy(df_test, forecast_final):
    """
    Displays accuracy metrics for all forecasted features listed in the global 'features' variable.

    Calls 'forecast_accuracy' for each feature and prints the results in a formatted table.

    Parameters:
        df_test (pd.DataFrame): DataFrame with actual observed values.
        forecast_final (pd.DataFrame): DataFrame containing forecasted values.
    """
    print("{:>18} {:>15} {: >25} {: >25}".format(*["Series", "Mean Error", "Mean Absolute Error", "Root Mean Squared Error"]))
    for col in features:
        forecast_accuracy(df_test, forecast_final, col)

print("\n\n"+color.BOLD + color.PURPLE + 'New York:' + color.END)
display_accuracy(ny_test, ny_forecast_final)
print("\n\n"+color.BOLD + color.PURPLE + 'London:' + color.END)
display_accuracy(ldn_test, ldn_forecast_final)

## Logging Model to MLflow

In [ ]:
class TwoCitiesModel(mlflow.pyfunc.PythonModel):
    def load_context(self):
        try:    
            

        except Exception as e:
            logger.error(f"Error error during initialization: {str(e)}")
            raise
    
    def predict(self, context, model_input, params = None):
        """
        Computes the predicted class of Iris Flower.
        """
        try:
            
        except Exception as e:
            logger.error(f"Error performing prediction: {str(e)}")
            raise
    
    @classmethod
    def log_model(cls, model_name):
        """
        Logs the model to MLflow with appropriate artifacts and schema.
        """
        try:
            # Define input and output schema
            input_schema = Schema([
                ColSpec("string","citie"),
                
                ])
            output_schema = Schema([
                ColSpec("string", "class"),
            ])
            
            # Define model signature
            signature = ModelSignature(inputs=input_schema, outputs=output_schema)
            
            # Log the model in MLflow
            mlflow.pyfunc.log_model(
                model_name,
                python_model=cls(),
                signature=signature,

            )
        except Exception as e:
            logger.error(f"Error logging model: {str(e)}")
            raise

In [ ]:
logger.info(f'Starting the experiment: {EXPERIMENT_NAME}')

# Set the MLflow experiment name
mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)

# Start an MLflow run
with mlflow.start_run(run_name=RUN_NAME) as run:
    # Print the artifact URI for reference
    logging.info(f"Run's Artifact URI: {run.info.artifact_uri}")
    
    # Log the model to MLflow
    TwoCitiesModel.log_model(model_name=MODEL_NAME)

    # Register the logged model in MLflow Model Registry
    mlflow.register_model(
        model_uri=f"runs:/{run.info.run_id}/{MODEL_NAME}", 
        name=MODEL_NAME
    )

logger.info(f'Registered the model: {MODEL_NAME}')

## Fetching the Latest Model Version from MLflow

In [ ]:
# Initialize the MLflow client
client = MlflowClient()

# Retrieve the latest version of the "Iris_Flower_Model" model (not yet in a specific stage)
model_metadata = client.get_latest_versions(MODEL_NAME, stages=["None"])
latest_model_version = model_metadata[0].version  # Extract the latest model version

# Fetch model information, including its signature
model_info = mlflow.models.get_model_info(f"models:/{MODEL_NAME}/{latest_model_version}")

# Print the latest model version and its signature
print(f"Latest Model Version: {latest_model_version}")
print(f"Model Signature: {model_info.signature}")

## Loading the Model and Running Inference

In [ ]:
model = mlflow.pyfunc.load_model(model_uri=f"models:/{MODEL_NAME}/{latest_model_version}")
predictions = model.predict(x_test[:3])

print("Input:", x_test)
print("\nPrediction:", predictions)

In [ ]:
logger.info('Notebook execution completed.')

Built with ❤️ using [**Z by HP AI Studio**](https://zdocs.datascience.hp.com/docs/aistudio/overview).